# 1. 词向量训练语料的构建
训练集后一亿 + 测试集A + 测试集B

## 1.1 提取训练集后一亿语料
将query和title混合并利用set去重

In [2]:
import pandas as pd
import pickle
# 读取后一亿数据用于训练
train = pd.read_csv'/home/kesci/input/bytedance/test_final_part1.csv', header=None,
     names=['query_id', 'query', 'query_title_id', 'title'], nrows=100000000, skiprows=900000000)
     
train.to_csv('/home/kesci/work/train_data/train_final.csv', header=False, index=False)
b = []
f = open("/home/kesci/work/train_data/train_final.csv")
lines = f.readlines(256*1024*1024)
count = 0
while lines:
    for line in lines:
        count += 1
        line = line.strip().split(",")
        query = line[1]
        title = line[3]
        b.append(query)
        b.append(title)
        if count % 10000000 == 0:
            print("count ", count)
    lines = f.readlines(256*1024*1024)
f.close()
b = set(b)

## 1.2 提取测试集语料
将query和title混合并利用set去重

In [ ]:
a = []
f = open("/home/kesci/input/bytedance/bytedance_contest.final_2.csv")
lines = f.readlines(256*1024*1024)
count = 0
while lines:
    for line in lines:
        count += 1
        line = line.strip().split(",")
        query = line[1]
        title = line[3]
        a.append(query)
        a.append(title)
        if count % 10000000 == 0:
            print("count ", count)
    lines = f.readlines(256*1024*1024)
f.close()
a = set(a)
c = []
f = open("/home/kesci/input/bytedance/test_final_part1.csv")
lines = f.readlines(256*1024*1024)
count = 0
while lines:
    for line in lines:
        count += 1
        line = line.strip().split(",")
        query = line[1]
        title = line[3]
        c.append(query)
        c.append(title)
        if count % 10000000 == 0:
            print("count ", count)
    lines = f.readlines(256*1024*1024)
f.close()
c = set(c)

## 1.3 将训练集和测试集语料混合在一起并去重

In [ ]:
d = (set(c) | set(b) | set(a))
# 将语料保存成gensim中LineSentence的格式
with open("silver_bullet/corpus", mode="w") as f:
    for i in d:
        f.write(i + '\n')
with open("silver_bullet/final_set.pickle", mode="wb") as f:
    pickle.dump(d, f)

# 2. 词向量的训练
根据步骤1提取的语料分别训练word2vec和fasttext

In [ ]:
import gensim
import time
import pandas as pd
import numpy as np
from gensim.models import word2vec,fasttext
from gensim.models.fasttext import FastText
from gensim.test.utils import get_tmpfile, datapath
WV_MODEL_PATH = "/home/kesci/work/helper_model/w2v_final2.model"
FAST_MODEL_PATH = "/home/kesci/work/helper_model/fasttext_final.model"

## 2.1 训练word2vec
word2vec每个epoch监控一下loss，在第8个epoch的loss异常的变成了0，多次训练都是类似的结果(可能跟数据量大小有关)最终取了第7个epoch个word2vec


In [ ]:
# 后面加载训练好的w2v模型时也需要有这个类的定义, 否则load会报找不到这个类的错误
class EpochSaver(gensim.models.callbacks.CallbackAny2Vec):
    '''词向量训练的回调函数，用于保存模型，打印损失函数等等'''
    def __init__(self, save_name):
        self.save_path = save_name
        self.epoch = 0
        self.pre_loss = 0
        self.best_loss = 999999999.9
        self.since = time.time()

    def on_epoch_end(self, model):
        self.epoch += 1
        cum_loss = model.get_latest_training_loss() # 返回的是从第一个epoch累计的
        epoch_loss = cum_loss - self.pre_loss
        time_taken = time.time() - self.since
        print("Epoch %d, loss: %.2f, time: %dmin %ds" %
                    (self.epoch, epoch_loss, time_taken//60, time_taken%60))
        # 避免loss等于0的情况
        if self.best_loss > epoch_loss and epoch_loss > 0:
            self.best_loss = epoch_loss
            print("Better model. Best loss: %.2f" % self.best_loss)
            model.save(self.save_path)
            print("Model %s save done!" % self.save_path)

        self.pre_loss = cum_loss
        self.since = time.time()
# 训练300维的word2vec
model_word2vec = gensim.models.Word2Vec(min_count=2,window=5,size=300,workers=15,batch_words=500000)
linesen = datapath('/home/kesci/work/silver_bullet/corpus') # 语料文件
model_word2vec.build_vocab(corpus_file=linesen)
model_word2vec.train(corpus_file=linesen, total_words=model_word2vec.corpus_total_words, 
        total_examples=model_word2vec.corpus_count,epochs=30, compute_loss=True, report_delay=60,
callbacks=[EpochSaver("/home/kesci/work/helper_model/w2v_final2.model")])

## 2.2 训练fasttext
由于时间关系，fasttext只训练了20个epoch

In [ ]:
class EpochSaver(gensim.models.callbacks.CallbackAny2Vec):
     "Callback to save model after every epoch"
     def __init__(self, path_prefix, epochs=0):
         self.path_prefix = path_prefix
         self.epoch = epochs

     def on_epoch_end(self, model):
         output_path = self.path_prefix
         print("Save model to {}".format(output_path))
         model.save(output_path)
         self.epoch += 1
# 训练300维的fasttext词向量
ft = FastText(size=300,window=5,min_count=2,workers=15,batch_words=200000)
linesen = datapath('/home/kesci/work/silver_bullet/corpus')
ft.build_vocab(corpus_file=linesen)
ft.train(corpus_file=linesen, total_words=ft.corpus_total_words,
total_examples=ft.corpus_count,epochs=30, compute_loss=True,
callbacks=[EpochSaver("/home/kesci/work/helper_model/fasttext_final.model")])

# 3. tfidfvectorizer和countVectorizer模型
后续将使用tfidf, cv计算相似度等特征

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pickle
with open("silver_bullet/final_set.pickle", mode="rb") as f:
    d = pickle.load(f)
# d:set 包含去重后的所有query, title
TF_IDF_MODEL_PATH = "helper_model/tf_idf.model"
CV_MODEL_PATH = "helper_model/countVectorizer.model"

# tf_idf
tf_idf = TfidfVectorizer(min_df=2,use_idf=True).fit(d)
with open(TF_IDF_MODEL_PATH, 'wb') as f:
    pickle.dump(tf_idf, f)

# cv
cv = CountVectorizer(max_df=0.01, min_df=2).fit(d)
with open(CV_MODEL_PATH, 'wb') as f:
    pickle.dump(cv, f)

# 4. 特征工程

In [ ]:
import os
import gc
import csv
import time
import math
import random
import pickle
import inspect
from gensim.models import word2vec,fasttext
import gensim
import gzip
import multiprocessing
from collections.abc import Iterable
from scipy import spatial
from sklearn.metrics.pairwise import *
import sklearn.preprocessing as pp
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
tqdm.pandas()

## 4.1 nuniuqe全局特征

In [ ]:
# 分别读取训练集的最后一亿，测试集A，测试集B
train = pd.read_csv('/home/kesci/work/train_data/train_final.csv', header=None, 
        nrows=int(1e8), skiprows=int(9e8))
test1 = pd.read_csv('/home/kesci/input/bytedance/test_final_part1.csv', header=None)
test2 = pd.read_csv('/home/kesci/input/bytedance/bytedance_contest.final_2.csv', header=None)
dt = pd.concat([train, test1, test2])
dt.columns = [0, 'query', 2, 'title', 4, 5, 6]
dt = dt[['query', 'title']]

# 计算query, title的unique的个数
dt['query_nunique_title'] = dt.groupby('query').title.transform('nunique')
dt['title_nunique_query'] = dt.groupby('title').query.transform('nunique')

train_number = 100000000
test1_number = 20000000
test2_number = 100000000

train_nuinque = dt[['query_nunique_title', 'title_nunique_query']][:train_number]
with open("/home/kesci/work/train_data/train_final_nunique.pickle", mode="wb") as f:
    pickle.dump(train_nuinque, f)
test1_nuinque = dt[['query_nunique_title', 'title_nunique_query']][train_number:train_number+test1_number]
with open("/home/kesci/work/testdata1/test1_nunique.pickle", mode="wb") as f:
    pickle.dump(test1_nuinque, f)    
test2_nuinque = dt[['query_nunique_title', 'title_nunique_query']][train_number+test1_number:]
with open("/home/kesci/work/testdata2/test2_nunique.pickle", mode="wb") as f:
    pickle.dump(test2_nuinque, f)    

## 4.2 tfidf和cv相似度特征

In [ ]:
import pickle
TF_IDF_MODEL_PATH = "/home/kesci/work/helper_model/tf_idf.model"
CV_MODEL_PATH = "/home/kesci/work/helper_model/countVectorizer.model"
with open(TF_IDF_MODEL_PATH, mode="rb") as f:
    tfidf = pickle.load(f)
with open(CV_MODEL_PATH, mode="rb") as f:
    cv = pickle.load(f)

In [ ]:
# 对query和title进行count vectorize向量化
cv_query = cv.transform(dt['query'])
cv_title = cv.transform(dt['title'])
number = 10000
epochs = int(cv_query.shape[0] / number)
for i in range(epochs):
    cv_query1 = cv_query[i*number:(i+1)*number]
    cv_title1 = cv_title[i*number:(i+1)*number]
    # 计算query，title向量化后的余弦，欧几里得，曼哈顿距离
    c = paired_cosine_distances(cv_query1, cv_title1)
    e = paired_euclidean_distances(cv_query1, cv_title1)
    m = paired_manhattan_distances(cv_query1, cv_title1)
    if i == 0:
        a1 = c
        a2 = e
        a3 = m
    else:
        a1 = np.concatenate((a1,c))
        a2 = np.concatenate((a2,e))
        a3 = np.concatenate((a3,m))
cv_feature = np.array([a1,a2,a3]).T

train_cv = cv_feature[:train_number]
with open('/home/kesci/work/train_data/train_final_cv', mode="wb") as f:
    pickle.dump(train_cv, f)
test1_cv = cv_feature[train_number:train_number+test1_number]
with open("/home/kesci/work/testdata1/test1_final_cv", mode="wb") as f:
    pickle.dump(test1_cv, f)    
test2_cv = cv_feature[train_number+test1_number:]
with open("testdata2/test2_final_cv", mode="wb") as f:
    pickle.dump(test2_cv, f)

In [ ]:
# 对query和title进行tfidf vectorize向量化
tf_idf_query = tfidf.transform(dt['query'])
tf_idf_title = tfidf.transform(dt['title'])
number = 10000
epochs = int(tf_idf_query.shape[0] / number)
for i in range(epochs):
    tf_idf_query1 = cv_query[i*number:(i+1)*number]
    tf_idf_title1 = cv_title[i*number:(i+1)*number]
    # 计算query，title向量化后的余弦，欧几里得，曼哈顿距离
    c = paired_cosine_distances(tf_idf_query1, tf_idf_title1
    e = paired_euclidean_distances(tf_idf_query1, tf_idf_title1)
    m = paired_manhattan_distances(tf_idf_query1, tf_idf_title1)
    if i == 0:
        a1 = c
        a2 = e
        a3 = m
    else:
        a1 = np.concatenate((a1,c))
        a2 = np.concatenate((a2,e))
        a3 = np.concatenate((a3,m))
tf_idf_feature = np.array([a1,a2,a3]).T

train_tfidf = tf_idf_feature[:train_number]
with open('/home/kesci/work/train_data/train_final_tfidf', mode="wb") as f:
    pickle.dump(train_tfidf, f)
test1_tfidf = tf_idf_feature[train_number:train_number+test1_number]
with open('/home/kesci/work/testdata1/test1_final_tfidf', mode="wb") as f:
    pickle.dump(test1_tfidf, f)    
test2_tfidf = tf_idf_featuree[train_number+test1_number:]
with open('/home/kesci/work/testdata2/test2_final_tfidf', mode="wb") as f:
    pickle.dump(test2_tfidf, f)

## 4.3 其余特征
诸如长度特征、是否存在、编辑距离、jarrard距离、词向量相似度等特征都写在Feature类下面（具体解释在文档中），方便多线程加速。

In [ ]:
tfidf_dic = dict(zip(tfidf.get_feature_names(), tfidf.idf_))
tfidf_set = tfidf.get_feature_names()

class EpochSaver(gensim.models.callbacks.CallbackAny2Vec):
    '''用于保存模型, 打印损失函数等等'''
    def __init__(self, save_name):
        self.save_path = save_name
        self.epoch = 0
        self.pre_loss = 0
        self.best_loss = 999999999.9
        self.since = time.time()

    def on_epoch_end(self, model):
        self.epoch += 1
        cum_loss = model.get_latest_training_loss() # 返回的是从第一个epoch累计的
        epoch_loss = cum_loss - self.pre_loss
        time_taken = time.time() - self.since
        print("Epoch %d, loss: %.2f, time: %dmin %ds" %
                    (self.epoch, epoch_loss, time_taken//60, time_taken%60))
        if self.best_loss > epoch_loss:
            self.best_loss = epoch_loss
            print("Better model. Best loss: %.2f" % self.best_loss)
            model.save(self.save_path)
            print("Model %s save done!" % self.save_path)

        self.pre_loss = cum_loss
        self.since = time.time()

# 载入词向量模型
w2v = word2vec.Word2Vec.load(WV_MODEL_PATH)
ft = fasttext.FastText.load(FAST_MODEL_PATH)

In [ ]:
class Feature:
    '''
    逐行进行特征工程，静态方法用于输出特征。
    '''

    def __init__(self, skip_rows=0):
        # 以后做全局特征的时候可能要用
        self.idx = skip_rows
        # 包括了所有的静态方法的名字
        self._static = [i for i in dir(self)
                        if isinstance(inspect.getattr_static(self, i), staticmethod)]

    def output(self, s1, s2):
        '''输出所有静态方法下的特征'''
        self.idx += 1
        # 逐一调用每个静态方法
        # output = [getattr(self, method)(s1.split(" "), s2.split(" ")) for method in self._static]
        output = []
        s1, s2 = s1.split(), s2.split()
        # 逐一导入各个特征
        output.append(self.length_feature(s1, s2))
        output.append(self.isin(s1, s2))
        output.append(self.lev(s1, s2))
        output.append(self.Jaccrad(s1, s2))
        output.append(self.longestCommonSubsequence(s1, s2))
        output.append(self.lcsubstr_lens(s1, s2))
        output.append(self.prefix_suffix_features(s1, s2))
        return list(Feature.flatten(output))

    @classmethod
    def flatten(cls, l: Iterable):
        '''列表铺平'''
        for i in l:
            if isinstance(i, Iterable):
                yield from i
            else:
                yield i

    # ----- 长度特征 ------
    @staticmethod
    def length_feature(q1_tokens, q2_tokens):
        q1_unique_tokens = set(q1_tokens)
        q2_unique_tokens = set(q2_tokens)
        shared_tokens = q1_unique_tokens.intersection(q2_unique_tokens)
        q1_token_count = len(q1_tokens)
        q2_token_count = len(q2_tokens)
        q1_unique_token_count = len(q1_unique_tokens)
        q2_unique_token_count = len(q2_unique_tokens)
        q1_unique_ratio = q1_unique_token_count / q1_token_count
        q2_unique_ratio = q2_unique_token_count / q2_token_count
        min_token_count = min(q1_token_count, q2_token_count)
        max_token_count = max(q1_token_count, q2_token_count)
        min_unique_token_count = min(q1_unique_token_count, q2_unique_token_count)
        max_unique_token_count = max(q1_unique_token_count, q2_unique_token_count)
        min_unique_ratio = min(q1_unique_ratio, q2_unique_ratio)
        max_unique_ratio = max(q1_unique_ratio, q2_unique_ratio)
        count_add = q1_token_count + q2_token_count
        count_sub = abs(q1_token_count - q2_token_count)
        count_mul = q1_token_count * q2_token_count
        unique_count_add = q1_unique_token_count + q2_unique_token_count
        unique_count_sub = abs(q1_unique_token_count - q2_unique_token_count)
        unique_count_mul = q1_unique_token_count * q2_unique_token_count
        unique_ratio_add = q1_unique_ratio + q2_unique_ratio
        unique_ratio_sub = abs(q1_unique_ratio - q2_unique_ratio)
        unique_ratio_mul = q1_unique_ratio * q2_unique_ratio
        shared_ratio_min = len(shared_tokens) / max(q1_unique_token_count, q2_unique_token_count)
        shared_ratio_max = len(shared_tokens) / min(q1_unique_token_count, q2_unique_token_count)
        return [min_token_count, max_token_count,
          min_unique_token_count, max_unique_token_count,
          min_unique_ratio, max_unique_ratio,
         count_add, count_sub,
          unique_count_add, unique_count_sub, unique_count_mul,
          unique_ratio_add, unique_ratio_sub, unique_ratio_mul,
          shared_ratio_min, shared_ratio_max]


    # ------是否存在------
    @staticmethod
    def isin(s1, s2):
        return [int(' '.join(s1) in ' '.join(s2)), np.mean([int(w in s2) for w in s1])]


    # ------ 编辑距离 ------
    @staticmethod
    def lev(s1, s2):
        n1, n2 = len(s1), len(s2)
        lev_matrix = [[0 for i1 in range(n1 + 1)] for i2 in range(n2 + 1)]
        for i1 in range(1, n1 + 1):
            lev_matrix[0][i1] = i1
        for i2 in range(1, n2 + 1):
            lev_matrix[i2][0] = i2
        for i2 in range(1, n2 + 1):
            for i1 in range(1, n1 + 1):
                cost = 0 if s1[i1 - 1] == s2[i2 - 1] else 1
                elem = min(lev_matrix[i2 - 1][i1] + 1,
                           lev_matrix[i2][i1 - 1] + 1,
                           lev_matrix[i2 - 1][i1 - 1] + cost)
                lev_matrix[i2][i1] = elem
        total_length = s1.__len__() + s2.__len__()
        return [lev_matrix[-1][-1], (total_length - lev_matrix[-1][-1]) / total_length]
    
    # ------ jaccrad相似度 ------  
    @staticmethod
    def Jaccrad(s1, s2):
        grams_reference = set(s1)
        grams_model = set(s2)
        temp = 0
        for i in grams_reference:
            if i in grams_model:
                temp = temp + 1
        fenmu = len(grams_model) + len(grams_reference) - temp  # 并集
        jaccard_coefficient = float(temp / fenmu)  # 交集
        del temp
        del fenmu
        del grams_model
        del grams_reference
        return jaccard_coefficient


    # ------ 最长公共子序列 ------ 
    @staticmethod
    def longestCommonSubsequence(s1, s2):
        # 生成字符串长度加1的0矩阵，m用来保存对应位置匹配的结果
        m = [[0 for x in range(len(s2) + 1)] for y in range(len(s1) + 1)]
        # d用来记录转移方向
        d = [[None for x in range(len(s2) + 1)] for y in range(len(s1) + 1)]
        for p1 in range(len(s1)):
            for p2 in range(len(s2)):
                if s1[p1] == s2[p2]:  # 字符匹配成功，则该位置的值为左上方的值加1
                    m[p1 + 1][p2 + 1] = m[p1][p2] + 1
                    d[p1 + 1][p2 + 1] = 'ok'
                elif m[p1 + 1][p2] > m[p1][p2 + 1]:  # 左值大于上值，则该位置的值为左值，并标记回溯时的方向
                    m[p1 + 1][p2 + 1] = m[p1 + 1][p2]
                    d[p1 + 1][p2 + 1] = 'left'
                else:  # 上值大于左值，则该位置的值为上值，并标记方向up
                    m[p1 + 1][p2 + 1] = m[p1][p2 + 1]
                    d[p1 + 1][p2 + 1] = 'up'
        (p1, p2) = (len(s1), len(s2))
        s = []
        while m[p1][p2]:  # 不为None时
            c = d[p1][p2]
            if c == 'ok':  # 匹配成功，插入该字符，并向左上角找下一个
                s.append(s1[p1 - 1])
                p1 -= 1
                p2 -= 1
            if c == 'left':  # 根据标记，向左找下一个
                p2 -= 1
            if c == 'up':  # 根据标记，向上找下一个
                p1 -= 1
        del m
        del d
        return [len(s), len(s)/max(len(s1), len(s2)), len(s)/min(len(s1), len(s2))]
    
    # ------ 最长匹配的长度 ------
    @staticmethod
    def lcsubstr_lens(s1, s2):
        m=[[0 for i in range(len(s2)+1)]  for j in range(len(s1)+1)]  #生成0矩阵，为方便后续计算，比字符串长度多了一列
        mmax=0   #最长匹配的长度
        p=0  #最长匹配对应在s1中的最后一位
        for i in range(len(s1)):
            for j in range(len(s2)):
                if s1[i]==s2[j]:
                    m[i+1][j+1]=m[i][j]+1
                    if m[i+1][j+1]>mmax:
                        mmax=m[i+1][j+1]
                        p=i+1
        del m
        return [mmax, mmax/max(len(s1), len(s2)), mmax/min(len(s1), len(s2))]
    
    # ------ 前缀后缀特征 ------
    @staticmethod
    def prefix_suffix_features(q1_tokens, q2_tokens):
        q1_tokens_reversed, q2_tokens_reversed = q1_tokens[::-1], q2_tokens[::-1]
        max_score = max(len(q1_tokens), len(q2_tokens))
        max_recip_score = sum([1 / pos for pos in range(1, max_score + 1)])

        prefix_score, recip_prefix_score, idf_prefix_score = 0.0, 0.0, 0.0
        for i, (tok1, tok2) in enumerate(zip(q1_tokens, q2_tokens)):
            if tok1 == tok2:
                prefix_score += 1.0
                recip_prefix_score += 1 / (i + 1)
                # idf_prefix_score += token2idf[tok1]
            else:
                prefix_score /= max_score
                recip_prefix_score /= max_recip_score
                # idf_prefix_score /= max_idf_score

        suffix_score, recip_suffix_score, idf_suffix_score = 0.0, 0.0, 0.0
        for i, (tok1, tok2) in enumerate(zip(q1_tokens_reversed, q2_tokens_reversed)):
            if tok1 == tok2:
                suffix_score += 1.0
                recip_suffix_score += 1 / (i + 1)
            else:
                suffix_score /= max_score
                recip_suffix_score /= max_recip_score
                
        return [prefix_score, recip_prefix_score,
                suffix_score, recip_suffix_score]
                
    @classmethod
    def fasttext_transform(cls, sentence):
        '''将句子的每个词用fasttext embedding转化成向量，计算这些向量的平均值'''
        size = ft.trainables.layer1_size
        length = len(sentence)
        vec = np.zeros(shape=(1, size), dtype=np.float32)
        for word in sentence:
            try:
                vec += ft.wv[word] * tfidf_dic[word]
            # 如果这个词不在vocabulary中
            except:
                length -= 1
        return vec / length if length > 0 else vec
    
    @classmethod
    def w2v_transform(cls, sentence):
        '''将句子的每个词用word2vec embedding转化成向量，计算这些向量的平均值'''
        size = w2v.trainables.layer1_size
        length = len(sentence)
        vec = np.zeros(shape=(1, size), dtype=np.float32)
        for word in sentence:
            try: 
                vec += w2v.wv[word] * tfidf_dic[word]
            # 如果这个词不在vocabulary中
            except:
                length -= 1
        return vec / length if length > 0 else vec
    
    # ------ 词向量相似度特征 ------
    @staticmethod
    def vecotor_compute_cosine(s1, s2):
        fv1 = Feature.fasttext_transform(s1)
        fv2 = Feature.fasttext_transform(s2)
        fv3 = Feature.w2v_transform(s1)
        fv4 = Feature.w2v_transform(s2)
        a = [
                spatial.distance.braycurtis(fv1, fv2),spatial.distance.braycurtis(fv3, fv4),
                spatial.distance.canberra(fv1, fv2),spatial.distance.canberra(fv3, fv4),
                spatial.distance.chebyshev(fv1, fv2),spatial.distance.chebyshev(fv3, fv4),
                spatial.distance.correlation(fv1, fv2),spatial.distance.correlation(fv3, fv4),
                spatial.distance.cosine(fv1, fv2),spatial.distance.cosine(fv3, fv4),
                spatial.distance.euclidean(fv1, fv2),spatial.distance.euclidean(fv3, fv4),
                ]
        return a

In [ ]:
feature = Feature()
def func(df):
    return df.progress_apply(lambda x: feature.output(x['query'], x['title']), axis = 1)
n_processes = 15
# 将dataframe分成几部分，用multiprocessing模块下的进程池加速apply的过程
data_split = np.array_split(df, n_processes)
pool = multiprocessing.Pool(n_processes)
c = pd.concat(pool.map(func, data_split)) # 聚合多个进程的apply结果
discrete_features = np.asarray(c.tolist())
for i in range(discrete_features.shape[1]):
    discrete_features[discrete_features[:, i] == None, i] = 0
discrete_features = pd.DataFrame(discrete_features)
discrete_features = discrete_features.fillna(0)
discrete_features = np.array(discrete_features)

In [ ]:
import h5py
# 定义加载和保存hdf5文件的方法
def load_hdf5(infile):
    with h5py.File(infile,"r") as f:
         return f["image"][()]
def write_hdf5(arr,outfile):
  with h5py.File(outfile,"w") as f:
    f.create_dataset("image", data=arr, dtype=arr.dtype)

In [ ]:
# 用hdf5格式保存得到的特征
write_hdf5(discrete_features[:train_number], '/home/kesci/work/train_data/train_feature')
write_hdf5(discrete_features[train_n\umber:train_number+test1_number], '/home/kesci/work/testdata1/test1_feature')
write_hdf5(discrete_features[train_number+test1_number:], '/home/kesci/work/testdata2/test2_feature')

## 4.4 pad_sequence，用于神经网络训练

In [ ]:
# 分别用长度15，30对query和title进行前向padding和向后trucating
# 保存数据
x1 = df['query'].str.split().values
x1 = keras.preprocessing.sequence.pad_sequences(x1, maxlen=15)
write_hdf5(x1[:train_number], "train_data/train_final_query")
write_hdf5(x1[train_n\umber:train_number+test1_number], "testdata1/testdata1_query")
write_hdf5(x1[train_number+test1_number:], "testdata2/test2_final_query")

x2 = df['title'].str.split().values
x2 = keras.preprocessing.sequence.pad_sequences(x2, maxlen=30)
write_hdf5(x2[:train_number], "train_data/train_final_title")
write_hdf5(x2[train_n\umber:train_number+test1_number], "testdata1/testdata1_title")
write_hdf5(x2[train_number+test1_number:], "testdata2/test2_final_title")

# 5. lightgbm模型
使用了3000W训练集，使用了步骤4中除了4.4之外的全部特征，进行5折训练

## 5.1 载入数据

In [ ]:
from scipy import sparse
import pandas as pd
import h5py
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gensim
import gzip
def load_hdf5(infile):
    with h5py.File(infile,"r") as f:
         return f["image"][()]

# 载入数据
TRAIN_TF_FEATURE_PATH = '/home/kesci/work/train_data/train_final_tfidf'
TRAIN_CV_FEATURE_PATH = '/home/kesci/work/train_data/train_final_cv'
TRAIN_QUERY_TITLE_PATH = '/home/kesci/work/train_data/train_final_nunique.pickle'
TRAIN_Y_PATH = '/home/kesci/work/train_data/train_final_y'

TEST_QUERY_TITLE_PATH = '/home/kesci/work/testdata2/test2_nunique.pickle'
TEST_TF_FEATURE_PATH = '/home/kesci/work/testdata2/test2_final_tfidf'
TEST_CV_FEATURE_PATH = '/home/kesci/work/testdata2/test2_final_cv'

with open(TRAIN_Y_PATH, mode="rb") as f:
    train_y = pickle.load(f)
with open(TRAIN_TF_FEATURE_PATH, mode="rb") as f:
    tf_idf_feature = pickle.load(f)
with open(TRAIN_CV_FEATURE_PATH, mode="rb") as f:
    tf_cv_feature = pickle.load(f)
with open(TRAIN_QUERY_TITLE_PATH, mode="rb") as f:
    query_title_feature = pickle.load(f)
train_y = [int(i) for i in train_y]
train_y = np.array(train_y)
feature = load_hdf5("/home/kesci/work/train_data/train_feature")
train_data = np.hstack((feature, tf_idf_feature,tf_cv_feature,query_title_feature))

## 5.2 五折训练

In [ ]:
NFOLDS = 5 # 设定5折
kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=2019)
kf = kfold.split(train_data, train_y)
# 超参数设置
params = {
    'boosting_type':'gbdt',
    'num_leaves':128,
    'reg_alpha':5,
    'reg_lambda':5,
    'n_estimators':4053,
    'objective':'binary',
    'subsample':0.7,
    'colsample_bytree':0.7,
    'subsample_freq':1,
    'learning_rate':0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'random_state':8012,
    'n_jobs':15,
    'metric':['auc','binary_logloss'],
    'verbosity':-1,
}
# 执行训练
for i, (train_fold, validate) in enumerate(kf):
    if i == 0:
        continue
    if i == 1:
        continue
    print(i)
    X_train, X_validate, label_train, label_validate = train_data[train_fold, :], train_data[validate, :], train_y[train_fold], train_y[validate]

    dtrain = lgb.Dataset(X_train, label_train)
    dvalid = lgb.Dataset(X_validate, label_validate, reference=dtrain)

    bst = lgb.train(params, dtrain, num_boost_round=10000, valid_sets=dvalid, 
    verbose_eval=10, early_stopping_rounds=100)
    # 保存训练好的模型
    with open("/home/kesci/work/5_lgb_cv" + str(i), mode="wb") as f:
        pickle.dump(bst, f)

## 5.3 lgb模型训练集预测
将预测所得的lgb5折特征也喂入神经网络

In [ ]:
b = np.zeros((len(train_data), 5))
for i in range(5):
    with open("/home/kesci/work/5_lgb_cv" + str(i), mode="rb") as f:
        gbm = pickle.load(f)
    b[:,i] = gbm.predict(train_data)

# 将预测的结果保存下来，作为神经网络的输入之一（模型stacking）
with open("/home/kesci/work/train_data/train_final_lgb_feature", mode="wb") as f:
    pickle.dump(b, f)

## 5.4 lgb模型测试集B预测

In [ ]:
with open(TEST_TF_FEATURE_PATH, mode="rb") as f:
    tf_idf_feature = pickle.load(f)
with open(TEST_CV_FEATURE_PATH, mode="rb") as f:
    tf_cv_feature = pickle.load(f)
with open(TEST_QUERY_TITLE_PATH, mode="rb") as f:
    query_title_feature = pickle.load(f)
feature = load_hdf5('/home/kesci/work/testdata2/test2_feature')
test_data = np.hstack((feature, tf_idf_feature,tf_cv_feature,query_title_feature))
b = np.zeros((len(test_data), 5))
for i in range(5):
    with open("/home/kesci/work/5_lgb_cv" + str(i), mode="rb") as f:
        gbm = pickle.load(f)
    b[:,i] = gbm.predict(test_data)
    
# 将预测的结果保存下来，作为神经网络的输入之一（模型stacking）
with open("/home/kesci/work/testdata2/test2_lgb_feature", mode="wb") as f:
    pickle.dump(b, f)

# 6. esim模型
最终提交的是两个esim模型，两个模型的主要区别是embedding词向量不同，一个是word2vec+fasttext加权平均，一个仅仅是word2vec

## 6.1 word2vec和fasttext提取embedding矩阵

In [ ]:
import numpy as np
import pickle

WV_MODEL_PATH = "/home/kesci/work/helper_model/w2v_final2.model"
FAST_MODEL_PATH = "/home/kesci/work/helper_model/fasttext_final.model"
WV_EMBEDDING_PATH1 = "features/word2vec_embedding1.pickle"
WV_EMBEDDING_PATH2 = "features/word2vec_embedding2.pickle"
FT_EMBEDDING_PATH1 = "features/fasttext_embedding1.pickle"
FT_EMBEDDING_PATH2 = "features/fasttext_embedding2.pickle"

class EpochSaver(gensim.models.callbacks.CallbackAny2Vec):
    "Callback to save model after every epoch"

    def __init__(self, path_prefix, epochs=0):
        self.path_prefix = path_prefix
        self.epoch = epochs

    def on_epoch_end(self, model):
        output_path = self.path_prefix
        print("Save model to {}".format(output_path))
        model.save(output_path)
        self.epoch += 1

fastvec_model = fasttext.FastText.load(FAST_MODEL_PATH)
wordvec_model = word2vec.Word2Vec.load(WV_MODEL_PATH)

n = 2000000
chunk_size = 1000000
'''生成w2v, ft两个词嵌入矩阵并保存'''
# ----- word2vec ------
w2_embeddings_matrix = np.zeros((n, wordvec_model.vector_size))
keys = wordvec_model.wv.vocab.keys()
for word in range(n):
    if str(word) in keys:
        w2_embeddings_matrix[word] = wordvec_model[str(word)]
with open(WV_EMBEDDING_PATH1, mode="wb") as f:
    pickle.dump(w2_embeddings_matrix[:chunk_size], f)
with open(WV_EMBEDDING_PATH2, mode="wb") as f:
    pickle.dump(w2_embeddings_matrix[chunk_size:], f)


# ----- fasttext ------
ft_embeddings_matrix = np.zeros((n, fastvec_model.vector_size))
keys = fastvec_model.wv.vocab.keys()
for word in range(n):
    if str(word) in keys:
        ft_embeddings_matrix[word] = fastvec_model[str(word)]
with open(FT_EMBEDDING_PATH1, mode="wb") as f:
    pickle.dump(ft_embeddings_matrix[:chunk_size], f)
with open(FT_EMBEDDING_PATH2, mode="wb") as f:
    pickle.dump(ft_embeddings_matrix[chunk_size:], f)

## 6.2 神经网络训练数据准备
两个esim的不同只是embedding_matrix的不同，以此来保证模型的多样性，提高融合收益

In [ ]:
import os
import gc
import csv
import time
import math
import random
import pickle
import inspect
import multiprocessing
from collections.abc import Iterable
from scipy import spatial

from scipy import sparse
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import lightgbm as lgb
from scipy import sparse
from gensim.models import word2vec,fasttext
import gensim
import gzip
from keras.callbacks import *
import numpy as np
from numpy import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from gensim.models import Word2Vec, FastText, word2vec
import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import *
from keras.layers.embeddings import Embedding
from keras.initializers import glorot_uniform
from keras.layers.noise import GaussianNoise
from keras import optimizers
from keras import regularizers
from keras.preprocessing.sequence import pad_sequences
from keras.activations import softmax
from keras.optimizers import Nadam, Adam
from keras.models import load_model
import pickle
from tqdm import tqdm
tqdm.pandas()
import pickle
from typing import List
import h5py
def load_hdf5(infile):
    with h5py.File(infile,"r") as f:
         return f["image"][()]
def write_hdf5(arr,outfile):
  with h5py.File(outfile,"w") as f:
    f.create_dataset("image", data=arr, dtype=arr.dtype)

In [ ]:
WV_EMBEDDING_PATH1 = "/home/kesci/work/embedding/word2vec_embedding1.pickle"
WV_EMBEDDING_PATH2 = "/home/kesci/work/embedding/word2vec_embedding2.pickle"
FT_EMBEDDING_PATH1 = "/home/kesci/work/embedding/fasttext_embedding1.pickle"
FT_EMBEDDING_PATH2 = "/home/kesci/work/embedding/fasttext_embedding2.pickle"

with open(WV_EMBEDDING_PATH1, mode="rb") as f:
    embed_matrix11 = pickle.load(f)
with open(WV_EMBEDDING_PATH2, mode="rb") as f:
    embed_matrix12 = pickle.load(f)
with open(FT_EMBEDDING_PATH1, mode="rb") as f:
    embed_matrix21 = pickle.load(f)
with open(FT_EMBEDDING_PATH2, mode="rb") as f:
    embed_matrix22 = pickle.load(f)
    
embed_matrix1 = np.vstack([embed_matrix11, embed_matrix12]).astype('float32')
embed_matrix2 = np.vstack([embed_matrix21, embed_matrix22]).astype('float32')

# 模型一的embedding_matrix
embed_matrix = embed_matrix2

# 模型二的embedding_matrix
embed_matrix = 0.5 * embed_matrix1 + 0.5 * embed_matrix2

In [ ]:
query = load_hdf5("train_data/train_final_query")
title = load_hdf5("train_data/train_final_title")
# 加载之前提取的特征
with open("/home/kesci/work/train_data/train_final_lgb_feature", mode="rb") as f:
    lgb_feature = pickle.load(f)
TRAIN_TF_FEATURE_PATH = '/home/kesci/work/train_data/train_final_tfidf'
TRAIN_CV_FEATURE_PATH = '/home/kesci/work/train_data/train_final_cv'
TRAIN_QUERY_TITLE_PATH = '/home/kesci/work/train_data/train_final_nunique.pickle'
TRAIN_Y_PATH = '/home/kesci/work/train_data/train_final_y'
with open(TRAIN_Y_PATH, mode="rb") as f:
    train_y = pickle.load(f)
with open(TRAIN_TF_FEATURE_PATH, mode="rb") as f:
    tf_idf_feature = pickle.load(f)
with open(TRAIN_CV_FEATURE_PATH, mode="rb") as f:
    tf_cv_feature = pickle.load(f)
with open(TRAIN_QUERY_TITLE_PATH, mode="rb") as f:
    query_title_feature = pickle.load(f)
train_y = [int(i) for i in train_y]
train_y = np.array(train_y)
train_data = np.hstack((tf_idf_feature,tf_cv_feature,query_title_feature,lgb_feature))

In [ ]:
X = np.array(range(len(train_data)))
y = np.zeros(len(train_data))
# 划分测试集和验证集
train_index, test_index, _, _ = train_test_split(X, y, test_size=0.01, random_state=42)
train_features = train_data[train_index, :]
val_features = train_data[test_index, :]
train_query = query[train_index,:]
val_query = query[test_index, :]
train_title = title[train_index,:]
val_title = title[test_index, :]
y_train = train_y[train_index]
y_val = train_y[test_index]

## 6.3 模型训练

In [ ]:
def auc(y_true, y_pred):
    '''计算auc的metric'''
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

# 以下是一些自定义的计算函数，用于神经网络的构建
def unchanged_shape(input_shape):
    return input_shape

def substract(input_1, input_2):
    neg_input_2 = Lambda(lambda x: -x, output_shape=unchanged_shape)(input_2)
    out_ = Add()([input_1, neg_input_2])
    return out_


def submult(input_1, input_2):
    "Get multiplication and subtraction then concatenate results"
    mult = Multiply()([input_1, input_2])
    sub = substract(input_1, input_2)
    out_ = Concatenate()([sub, mult])
    return out_


def apply_multiple(input_, layers):
    "Apply layers to input then concatenate result"
    if not len(layers) > 1:
        raise ValueError('Layers list should contain more than 1 layer')
    else:
        agg_ = []
        for layer in layers:
            agg_.append(layer(input_))
        out_ = Concatenate()(agg_)
    return out_


def time_distributed(input_, layers):
    out_ = []
    node_ = input_
    for layer_ in layers:
        node_ = TimeDistributed(layer_)(node_)
    out_ = node_
    return out_


def soft_attention_alignment(input_1, input_2):
    attention = Dot(axes=-1)([input_1, input_2])

    w_att_1 = Lambda(lambda x: softmax(x, axis=1),
                     output_shape=unchanged_shape)(attention)
    # 这里相当于矩阵转置了
    w_att_2 = Permute((2, 1))(Lambda(lambda x: softmax(x, axis=2),
                                     output_shape=unchanged_shape)(attention))

    in1_aligned = Dot(axes=1)([w_att_1, input_1])
    in2_aligned = Dot(axes=1)([w_att_2, input_2])
    return in1_aligned, in2_aligned


class AttentivePoolingLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentivePoolingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # 定义需要学习的参数
        if isinstance(input_shape, list):
            input_shape = input_shape[0]
        feature_dim = input_shape[-1]
        self.U = self.add_weight(shape=(feature_dim, feature_dim), name='U', initializer='uniform', trainable=True)
        self.built = True
        super(AttentivePoolingLayer, self).build(input_shape)

    def compute_output_shape(self, input_shape):
        feature_dim = input_shape[0][-1]
        return [(input_shape[0][0], feature_dim),(input_shape[1][0], feature_dim)]
        # return input_shape

    def call(self, inputs, **kwargs):
        q = inputs[0]
        a = inputs[1]

        time_step = inputs[0].shape[1].value
        feature_dim = inputs[0].shape[2].value

        G = tf.tensordot(q, self.U, axes=1)
        G = tf.reshape(G, shape=(-1, time_step, feature_dim), )
        G = tf.matmul(G, a, transpose_b=True)
        G = tf.nn.tanh(G)

        g_q = tf.reduce_max(G, axis=-1)
        g_a = tf.reduce_max(G, axis=1)

        g_q = tf.expand_dims(g_q, axis=-1)
        g_a = tf.expand_dims(g_a, axis=-1)

        g_q = tf.nn.softmax(g_q, dim=1)
        g_a = tf.nn.softmax(g_a, dim=1)

        # q = q * g_q
        # a = a * g_a

        q = tf.matmul(q, g_q, transpose_a=True)
        q = tf.squeeze(q, [-1])
        a = tf.matmul(a, g_a, transpose_a=True)
        a = tf.squeeze(a, [-1])

        return [q, a]

**模型的一些超参数**

In [ ]:
dense_dim=200
lstm_num = 30
lstm_drop = 0.3
BATCH_SIZE = 2000
MAX_SEQUENCE_LENGTH1 = 15
MAX_SEQUENCE_LENGTH2 = 30
EMBEDDING_DIM = 300
EPOCHS = 10
dense_dropout = 0.3
num_heads=3
units=20
spatial_dropout=0.3

**模型构建**

In [ ]:
q1 = Input(name='q1', shape=(MAX_SEQUENCE_LENGTH1,))
q2 = Input(name='q2', shape=(MAX_SEQUENCE_LENGTH2,))

# Embedding
eb_layer = Embedding(input_dim=embed_matrix.shape[0], output_dim=EMBEDDING_DIM,
        weights=[embed_matrix], trainable=False)
bn = BatchNormalization(axis=2)
q1_embed = bn(eb_layer(q1))
q2_embed = bn(eb_layer(q2))

# Encode
encode = Bidirectional(CuDNNLSTM(30, return_sequences=True))
q1_encoded = encode(q1_embed)
q2_encoded = encode(q2_embed)

# Attention
q1_aligned, q2_aligned = soft_attention_alignment(q1_encoded, q2_encoded)

# Compose
q1_combined = Concatenate()([q1_encoded, q2_aligned, submult(q1_encoded, q2_aligned)])
q2_combined = Concatenate()([q2_encoded, q1_aligned, submult(q2_encoded, q1_aligned)])

compose = Bidirectional(CuDNNLSTM(30, return_sequences=True))
q1_compare = compose(q1_combined)
q2_compare = compose(q2_combined)

# Aggregate
q1_rep = apply_multiple(q1_compare, [GlobalAvgPool1D(), GlobalMaxPool1D()])
q2_rep = apply_multiple(q2_compare, [GlobalAvgPool1D(), GlobalMaxPool1D()])

# magic featurues
magic_input = Input(shape=(13,))
magic_dense = BatchNormalization()(magic_input)
magic_dense = Dense(64, activation='relu')(magic_dense)
magic_dense = Dropout(0.3)(magic_dense)

# Classifier
merged = Concatenate()([q1_rep, q2_rep, magic_dense])
x = merged
dense = BatchNormalization()(merged)
dense = Dense(dense_dim, activation='elu')(dense)
dense = BatchNormalization()(dense)
dense = Dropout(dense_dropout)(dense)
dense = Dense(dense_dim, activation='elu')(dense)
dense = BatchNormalization()(dense)
dense = Dropout(dense_dropout)(dense)
out_ = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[q1, q2, magic_input], outputs=out_)
model.compile(optimizer=Adam(lr=1e-3), loss='binary_crossentropy', metrics=[auc])

**回调函数**

In [ ]:
class roc_callback(keras.callbacks.Callback):
    '''指数衰减学习率的回调函数'''
    def on_epoch_end(self, epoch, logs={}): 
        # 改变学习率
        old_lr = float(K.get_value(self.model.optimizer.lr))
        new_lr = old_lr * 0.9
        K.set_value(self.model.optimizer.lr, new_lr)
        print(f'学习率由{old_lr:.6f}调整为{new_lr:.6f}')
        return
    
roc_cb = roc_callback()
# 用验证集上的loss判断是否需要提前结束
cb= EarlyStopping(monitor="val_loss", patience=4, verbose=1, mode='min')
# 及时保存模型
checkpoint = ModelCheckpoint(filepath='models/esim1.h5', monitor='val_loss',verbose=1,save_best_only='True',mode='min',period=1)

**fit**

In [ ]:
hist = model.fit([train_query, train_title, train_features], y_train, batch_size=4000, 
epochs=30 , verbose=1, shuffle = True, 
validation_data = ([val_query, val_title, val_features], y_val),
callbacks=[cb,checkpoint, roc_cb])

## 6.4 模型预测

In [ ]:
# 加载模型和数据
model1 = load_model("models/esim1.h5", custom_objects={'auc':auc, "softmax":softmax})
model2 = load_model("models/esim2.h5", custom_objects={'auc':auc, "softmax":softmax})
with open("/home/kesci/work/testdata2/test2_lgb_feature", mode="rb") as f:
    lgb_feature = pickle.load(f)
TEST_TF_FEATURE_PATH = '/home/kesci/work/testdata2/test2_final_tfidf'
TEST_CV_FEATURE_PATH = '/home/kesci/work/testdata2/test2_final_cv'
TEST_QUERY_TITLE_PATH = '/home/kesci/work/testdata2/test2_nunique.pickle'
with open(TEST_TF_FEATURE_PATH, mode="rb") as f:
    tf_idf_feature = pickle.load(f)
with open(TEST_CV_FEATURE_PATH, mode="rb") as f:
    tf_cv_feature = pickle.load(f)
with open(TEST_QUERY_TITLE_PATH, mode="rb") as f:
    query_title_feature = pickle.load(f)
test_data = np.hstack((tf_idf_feature,tf_cv_feature,query_title_feature,lgb_feature))

query = load_hdf5("testdata2/test2_final_query")
title = load_hdf5("testdata2/test2_final_title")

# 进行预测并保存结果
nn_pred1 = model1.predict([query, title, test_data], verbose=1)
nn_pred2 = model2.predict([query, title, test_data], verbose=1)

with open("testdata2/nn_pred1.pickle", mode="wb") as f:
    pickle.dump(nn_pred1, f)
with open("testdata2/nn_pred2.pickle", mode="wb") as f:
    pickle.dump(nn_pred2, f)

# 7. 模型融合与提交

In [ ]:
test = pd.read_csv('/home/kesci/input/bytedance/bytedance_contest.final_2.csv', header=None,
    names=['query_id', 'query', 'query_title_id', 'title'], 
    usecols=['query_id', 'query_title_id'])
with open("testdata2/nn_pred1.pickle", mode="rb") as f:
    nn_pred1 = pickle.load(f) # 第一个esim的预测结果
test['pred'] = nn_pred1
test.to_csv('submit_nn1', header=False, index=False)
with open("testdata2/nn_pred2.pickle", mode="rb") as f:
    nn_pred2 = pickle.load(f) # 第二个esim的预测结果
test['pred'] = nn_pred2
test.to_csv('submit_nn2', header=False, index=False)
sub1 = pd.read_csv('submit_nn1', header=None)
sub2 = pd.read_csv('submit_nn2', header=None)

sub1[2] = 0.53 * sub1[2] + 0.47 * sub2[2] # 两个esim模型的预测结果加权平均，这也是我们在测试集B上的最终提交
sub1[2] = sub1[2].astype('float32')
sub1.to_csv('submit_ronghe1.csv', header=False, index=False)
!https_proxy="http://klab-external-proxy" ./kesci_submit -file submit_ronghe1.csv -token 99ff95d5193957a0 -mode archive